In [18]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import defaultdict

In [2]:
url='https://ditie.mapbar.com/beijing_line/'
response=requests.get(url).text

In [3]:
soup=BeautifulSoup(response,'lxml')

In [4]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>北京地铁线路大全_北京地铁线路图_北京地铁 最新-图吧地铁</title>
<meta content="北京地铁线路图,北京地铁线路,北京地铁,北京地铁线路图 最新" name="keywords"/>
<meta content="【北京地铁线路大全】北京图吧地铁，提供最新北京所有地铁线路查询，真实北京地铁线路图等信息。" name="description"/>
<link href="//r.mapbar.com/ditie/css/style.css" rel="stylesheet" type="text/css"/>
<link href="//r.mapbar.com/ditie/css/base.css" rel="stylesheet" type="text/css"/>
<link href="//r.mapbar.com/ditie/css/style_time.css" rel="stylesheet" type="text/css"/>
<script src="//api.mapbar.com/api/mapbar31.3.js" type="text/javascript"></script>
<script src="//r.mapbar.com/ditie/js/jquery-1.9.1.min.js" type="text/javascript"></script>
<script src="//cbjs.baidu.com/js/m.js" type="text/javascript"></script>
<script type="text/javascript">
		BAIDU_CLB_preloadSlots("934189","934191","946594");
	</script>
<script src="//s.union.360.cn/100009.js" type="text/javascript"></script>
</head>
<body>
<input id="cncity" type="hidden" value="北京市"/>
<div class="

In [5]:
data=pd.DataFrame()
subway=soup.find_all('div',class_="station")
for sub in subway:
    route_name=sub.find('strong',class_="bolder").text
    routes=sub.find('ul')
    routes=routes.find_all('a')
    for route in routes:
        temp={'name':route.text,'site':route_name}
        data=data.append(temp,ignore_index=True)

In [6]:
data['city']='北京'

In [7]:
data

,name,site,city
0,四惠站,地铁八通线,北京
1,四惠东站,地铁八通线,北京
2,高碑店站,地铁八通线,北京
3,传媒大学站,地铁八通线,北京
4,双桥站,地铁八通线,北京
...,...,...,...
346,永丰站,地铁16号线,北京
347,永丰南站,地铁16号线,北京
348,西北旺站,地铁16号线,北京
349,马连洼站,地铁16号线,北京


In [8]:
def get_location(keyword,city):
    request_url='http://restapi.amap.com/v3/place/text?key=79331b455d57d7b99de0d6385651b2fb&keywords='+keyword+'&types=&city='+city+'&children=1&offset=1&page=1&extensions=all'
    response=requests.get(request_url)
    result=response.text
    pattern='"location":"(.*?),(.*?)"'
    place=re.findall(pattern,result)
    try:
        return place[0][0],place[0][1]
    except:
        return get_location(keyword.replace('站',""),city)
data['longtitude'],data['latitude']=None,None
for index,row in data.iterrows():
    longitude,latitude=get_location(row['name'],row['city'])
    print(row['name'],longitude,latitude)
    data.iloc[index]['longtitude']=longitude
    data.iloc[index]['latitude']=latitude


四惠站 116.498489 39.908279
四惠东站 116.514954 39.908051
高碑店站 115.858901 39.328727
传媒大学站 116.554931 39.909548
双桥站 116.569999 39.900706
管庄站 116.598786 39.908696
八里桥站 116.617844 39.906559
通州北苑站 116.636748 39.904070
果园站 116.646183 39.893199
九棵树站 116.662943 39.888520
梨园站 116.669083 39.883915
临河里站 116.679099 39.875796
土桥站 116.687215 39.871679
西二旗站 116.306169 40.052307
生命科学园站 116.294575 40.094705
朱辛庄站 116.313248 40.104775
巩华城站 116.293671 40.130836
沙河站 116.265245 40.125131
沙河高教园站 116.279991 40.164629
南邵站 116.287159 40.207766
北邵洼站 116.282712 40.221261
昌平东关站 116.262840 40.221475
昌平站 116.194657 40.190277
十三陵景区站 116.207894 40.239493
昌平西山口站 116.194237 40.244382
苏庄站 116.125346 39.723503
良乡南关站 116.140429 39.723478
良乡大学城西站 116.155903 39.723547
良乡大学城站 116.176119 39.722872
良乡大学城北站 116.183056 39.729827
广阳城站 116.180550 39.746964
篱笆房站 116.189793 39.760327
长阳站 116.212569 39.763968
稻田站 116.218561 39.794460
大葆台站 116.291802 39.807984
郭公庄站 116.302856 39.814140
宋家庄站 116.428645 39.843724
肖村站 116.447857 39.834035
小红门站 

In [9]:
data

,name,site,city,longtitude,latitude
0,四惠站,地铁八通线,北京,116.498489,39.908279
1,四惠东站,地铁八通线,北京,116.514954,39.908051
2,高碑店站,地铁八通线,北京,115.858901,39.328727
3,传媒大学站,地铁八通线,北京,116.554931,39.909548
4,双桥站,地铁八通线,北京,116.569999,39.900706
...,...,...,...,...,...
346,永丰站,地铁16号线,北京,116.237766,40.071247
347,永丰南站,地铁16号线,北京,116.247876,40.064807
348,西北旺站,地铁16号线,北京,116.258460,40.048344
349,马连洼站,地铁16号线,北京,116.274073,40.032358


In [10]:
def find_lowest_cost_node(costs):
    lowest_cost=float('inf')
    lowest_cost_node=None
    for node in costs:
        if node not in processed:
            if costs[node]<lowest_cost:
                lowest_cost=costs[node]
                lowest_cost_node=node
    return lowest_cost_node

In [11]:
def find_shortest_path():
    node=end
    shortest_path=[end]
    while parents[node]!=start:
        shortest_path.append(parents[node])
        node=parents[node]
    shortest_path.append(start)
    return shortest_path

In [12]:
def dijkstar():
    #找到最小开销节点
    node=find_lowest_cost_node(costs)
    print('当前cost最小的节点：',node)
    while node is not None:
        #获取节点目前cost 
        cost=costs[node]
        neighbors=graph[node]
        for neighbor in neighbors.keys():
            new_cost=cost+float(neighbors[neighbor])
            #通过node，是否更新cost
            if neighbor not in costs or new_cost< costs[neighbor]:
                costs[neighbor]=new_cost
                parents[neighbor]=node
        processed.append(node)
        node=find_lowest_cost_node(costs)
        print('当前最小节点:',node)
    shortest_path=find_shortest_path()
    shortest_path.reverse()
    return shortest_path

In [13]:
def compute_distance(longtitude1,latitude1,longtitude2,latitude2):
    url='http://restapi.amap.com/v3/distance?key=79331b455d57d7b99de0d6385651b2fb&origins='+str(longtitude1)+','+str(latitude1)+'&destination='+str(longtitude2)+','+str(latitude2)+'&type=1'
    response=requests.get(url).text
    pattern='"distance":"(.*?)","duration":"(.*?)"'
    result=re.findall(pattern,response)
    return result[0][0]

In [14]:
compute_distance(116.337581,39.993138,116.339941,39.976228)

'2646'

In [15]:
#创建图中两点之间最短距离
graph=defaultdict(dict)
for i in range(data.shape[0]-1):
    site1=data.iloc[i]['site']
    site2=data.iloc[i+1]['site']
    if site1==site2:
        longtitude1,latitude1=data.iloc[i]['longtitude'],data.iloc[i]['latitude']
        longtitude2,latitude2=data.iloc[i+1]['longtitude'],data.iloc[i+1]['latitude']
        name1,name2=data.iloc[i]['name'],data.iloc[i+1]['name']
        distance=compute_distance(longtitude1,latitude1,longtitude2,latitude2)
        graph[name1][name2]=distance
        graph[name2][name1]=distance


In [19]:
# start='五道口站'
# end='北京南站'
# # print(graph[start].keys())
# #从start到节点距离
costs={}
#储存根节点
parents ={}
#储存处理过的节点
processed=[]
#如果是根节点，就是空的
# parents[end]=None
def compute(site1,site2):
    global start,end, parents, costs,processed 
    start=site1
    end=site2
    parents[end]=None
    for node in graph[start].keys():
        costs[node]=float(graph[start][node])
        parents[node]=start
    costs[end]=float('inf')
        #计算最短路径
    shortest_path=dijkstar()
    return shortest_path
#算出离目的地和起始地最近的地铁站点
def get_nearst_subway(data,location):
    distance=float('inf')
    nearst=None
    for i in range(data.shape[0]):
        site1=data.iloc[i]['name']
        longtitude=float(data.iloc[i]['longtitude'])
        latitude=float(data.iloc[i]['latitude'])
        temp=(float(location[0])-longitude)**2+(float(location[1])-latitude)**2
        if temp<distance:
            distance=temp
            nearst=site1
    return site1

In [20]:
#如果起始站不是地铁站，得出离最近地铁站
def site_location(site1,site2):
    location1=get_location(site1,'北京')
    location2=get_location(site2,'北京')
    start=get_nearst_subway(data,location1)
    end=get_nearst_subway(data,location2)
    shortest_path=compute(start,end)
    if site1!=start:
        shortest_path.insert(0,site1)
    if site2!=end:
        shortest_path.append(site2)
    print(shortest_path)

In [17]:
if __name__=='__main__':
    site1='五道口站'
    site2='北京南站'
    shortest_path=compute(site1,site2)
    print(shortest_path)

当前cost最小的节点： 知春路站
当前最小节点: 西土城站
当前最小节点: 知春里站
当前最小节点: 五道口站
当前最小节点: 海淀黄庄站
当前最小节点: 上地站
当前最小节点: 中关村站
当前最小节点: 苏州街站
当前最小节点: 大钟寺站
当前最小节点: 人民大学站
当前最小节点: 北京大学东门站
当前最小节点: 巴沟站
当前最小节点: 牡丹园站
当前最小节点: 西二旗站
当前最小节点: 圆明园站
当前最小节点: 魏公村站
当前最小节点: 健德门站
当前最小节点: 西苑站
当前最小节点: 火器营站
当前最小节点: 西直门站
当前最小节点: 国家图书馆站
当前最小节点: 北土城站
当前最小节点: 北宫门站
当前最小节点: 新街口站
当前最小节点: 安贞门站
当前最小节点: 白石桥南站
当前最小节点: 奥体中心站
当前最小节点: 惠新西街南口站
当前最小节点: 生命科学园站
当前最小节点: 安华桥站
当前最小节点: 动物园站
当前最小节点: 龙泽站
当前最小节点: 安河桥北站
当前最小节点: 平安里站
当前最小节点: 车公庄站
当前最小节点: 积水潭站
当前最小节点: 花园桥站
当前最小节点: 马连洼站
当前最小节点: 西四站
当前最小节点: 车公庄西站
当前最小节点: 长春桥站
当前最小节点: 安德里北街站
当前最小节点: 回龙观站
当前最小节点: 灵境胡同站
当前最小节点: 北海北站
当前最小节点: 惠新西街北口站
当前最小节点: 鼓楼大街站
当前最小节点: 和平西桥站
当前最小节点: 芍药居站
当前最小节点: 白堆子站
当前最小节点: 阜成门站
当前最小节点: 朱辛庄站
当前最小节点: 慈寿寺站
当前最小节点: 车道沟站
当前最小节点: 奥林匹克公园站
当前最小节点: 太阳宫站
当前最小节点: 和平里北街站
当前最小节点: 安定门站
当前最小节点: 西北旺站
当前最小节点: 西单站
当前最小节点: 霍营站
当前最小节点: 雍和宫站
当前最小节点: 南锣鼓巷站
当前最小节点: 大屯路东站
当前最小节点: 西钓鱼台站
当前最小节点: 什刹海站
当前最小节点: 回龙观东大街站
当前最小节点: 森林公园南门站
当前最小节点: 光熙门站
当前最小节点: 宣武门站
当前最小节点: 复兴门站
当前最小节点: 永丰南站
当前最小节点: 育知路站
当前最小节点: 北沙滩站
当前